In [1]:
import pandas as pd
import altair as alt

xl_data = pd.read_csv("csxl_data.csv")
app_lab_data = pd.read_csv("app-lab-data.csv")

In [2]:
app_lab_data['date'] = pd.to_datetime(app_lab_data['date'])

app_lab_data['timeIn'] = pd.to_timedelta(app_lab_data['timeIn'])

app_lab_data['start'] = app_lab_data['date'] + app_lab_data['timeIn']

# Get the day of the week
app_lab_data['day_of_week'] = app_lab_data['start'].dt.day_name()

# Get the hour of the day in non-military time (also remove leading 0s, for example, 01pm -> 1pm)
app_lab_data['civilian_time'] = app_lab_data['start'].dt.strftime("%I %p").str.lstrip("0")

# Group reservations by the day of the week + hour of the day + count the number ofidfferent reservations
app_lab_reservations_per_hour = app_lab_data.groupby(["day_of_week", "civilian_time"]).size().reset_index(name="count")

# Filter out Saturday and Sunday from App Lab data (since XL does not take reservations on weekends)
app_lab_reservations_per_hour = app_lab_reservations_per_hour[app_lab_reservations_per_hour['day_of_week'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

app_lab_reservations_per_hour.head(5)

,day_of_week,civilian_time,count
0,Friday,1 PM,11
1,Friday,10 AM,1
2,Friday,11 AM,2
3,Friday,12 PM,13
4,Friday,2 PM,12


In [3]:
xl_data['start'] = pd.to_datetime(xl_data['start'])

# Get the day of the week
xl_data['day_of_week'] = xl_data['start'].dt.day_name()

# Get the hour of the day in non-military time (also remove leading 0s, for example, 01pm -> 1pm)
xl_data['civilian_time'] = xl_data['start'].dt.strftime('%I %p').str.lstrip("0")

# Group reservations by the day of the week + hour of the day + count the number of different reservations
xl_reservations_per_hour = xl_data.groupby(['day_of_week', 'civilian_time']).size().reset_index(name='count')

xl_reservations_per_hour.head(5)

,day_of_week,civilian_time,count
0,Friday,1 PM,110
1,Friday,10 AM,85
2,Friday,11 AM,53
3,Friday,12 PM,90
4,Friday,2 PM,157


In [4]:
# Determine the maximum count for both datasets (for scaling the y axes)
max_count = max(app_lab_reservations_per_hour['count'].max(), xl_reservations_per_hour['count'].max())

# Create a domain for equal x axis
x_domain = ['9 AM', '10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM', '7 PM']

In [8]:
chart = alt.Chart(app_lab_reservations_per_hour).mark_bar().encode(
    x=alt.X('civilian_time:N', title='Hour of the Day', sort=x_domain),
    y=alt.Y('count:Q', title='Reservations', scale=alt.Scale(domain=[0, max_count])),
    column=alt.Column('day_of_week:N', title=None, sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']),
    tooltip=alt.Tooltip('count:Q')
).properties(
    height=150,
    title='Popular Times in the App Lab'
).configure_mark(color='#4786c6')

chart.show()

alt.Chart(...)

In [9]:
chart = alt.Chart(xl_reservations_per_hour).mark_bar().encode(
    x=alt.X('civilian_time:N', title='Hour of the Day', sort=x_domain, scale=alt.Scale(domain=x_domain)),
    y=alt.Y('count:Q', title='Reservations', scale=alt.Scale(domain=[0, max_count])),
    column=alt.Column('day_of_week:N', title=None, sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']),
    tooltip=alt.Tooltip('count:Q')
).properties(
    height=150,
    title='Popular Times in the CSXL'
).configure_mark(color='#4786c6')

chart.show()

alt.Chart(...)